In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
df_labels = pd.read_csv("/kaggle/input/dog-breed-identification/labels.csv")
print(df_labels)
df_labels['id'] = df_labels['id'] + '.jpg'
from sklearn.model_selection import train_test_split
df_train, df_validation = train_test_split(df_labels, test_size=0.2, shuffle=True)
df_train.shape, df_validation.shape

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
        rescale=1./255)

training_set = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory='/kaggle/input/dog-breed-identification/train',
    x_col="id",
    y_col="breed",
    target_size=(150, 150),
    class_mode="categorical",
    batch_size=128
)

validation_set = validation_datagen.flow_from_dataframe(
    dataframe=df_validation,
    directory='/kaggle/input/dog-breed-identification/train',
    x_col="id",
    y_col="breed",
    target_size=(150, 150),
    class_mode="categorical",
    batch_size=128
)

In [ ]:
classes = df_labels['breed'].unique().shape[0]
classes

In [ ]:
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=classes, activation='softmax'))

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.summary()

In [ ]:
# cnn.fit(x = training_set, validation_data = validation_set, epochs = 50)

In [ ]:
test_datagen = ImageDataGenerator ( 
    rescale=1/255. 
)

test_set = test_datagen.flow_from_directory (
    '/kaggle/input/dog-breed-identification',
    target_size = (150,150),
    batch_size = 128,
    classes=['test']
)

In [ ]:
# y_pred = cnn.predict(test_set)


In [ ]:
df_submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
df_submission

In [ ]:
import re
file_list = test_set.filenames
id_list = []
for name in file_list:
    m = re.sub('test/', '', name)
    m = re.sub('.jpg', '', m)
    id_list.append(m)

In [ ]:
id_list

In [ ]:
df_submission['id'] = id_list
df_submission.iloc[:,1:] = y_pred
df_submission

In [ ]:
final_df = df_submission.set_index('id')
final_df.to_csv('Submission_12.csv')